In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from imblearn import over_sampling, under_sampling
from imblearn.pipeline import Pipeline

In [ ]:
train_transaction_data = pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")
train_identity_data = pd.read_csv( "../input/ieee-fraud-detection/train_identity.csv")
test_transaction_data = pd.read_csv("../input/ieee-fraud-detection/test_transaction.csv")
test_identity_data = pd.read_csv("../input/ieee-fraud-detection/test_identity.csv")
submission=pd.read_csv("/kaggle/input/ieee-fraud-detection/sample_submission.csv")

In [ ]:
L=[train_transaction_data, train_identity_data, test_transaction_data,test_identity_data ]

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    #print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            #print("******************************")
            #print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            #print("dtype after: ",props[col].dtype)
            #print("******************************")
    
    # Print final result
    #print("___MEMORY USAGE AFTER COMPLETION:___")
    #mem_usg = props.memory_usage().sum() / 1024**2 
    #print("Memory usage is: ",mem_usg," MB")
    #print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props

for i in L:
    i= reduce_mem_usage(i)
   

In [ ]:
L=[train_transaction_data, train_identity_data, test_transaction_data,test_identity_data ]

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
train_transaction_data.head()


In [ ]:
train_transaction_data.shape

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
train_identity_data.head()

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
test_transaction_data.head()

In [ ]:

# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
test_identity_data.head()

In [ ]:
#train_transaction_data.head()

In [ ]:
#train_identity_data.head()

In [ ]:
#type(train_identity_data)

In [ ]:
#train_identity_data.describe()


In [ ]:
#print(train_identity_data.shape)
#print(train_transaction_data.shape)
#print(train_identity_data.nunique())
#print('')
#print(train_transaction_data.nunique())

In [ ]:
#features_train_identity=train_identity_data.columns
#features_train_transaction=train_transaction_data.columns


#print(train_identity_data[features_train_identity].isna().sum())
#print(train_transaction_data[features_train_transaction].isna().sum())

In [ ]:
#print(train_transaction_data.nunique())

In [ ]:
Train_Dataset_after_merge=pd.merge(train_transaction_data, train_identity_data, on="TransactionID",how='outer')
test_Dataset_after_merge=pd.merge(test_transaction_data, test_identity_data, on="TransactionID",how='outer')
type(test_Dataset_after_merge)

In [ ]:
import gc

In [ ]:
del(train_transaction_data)
del(train_identity_data)
del(test_transaction_data)
del(test_identity_data) 
gc.collect()
 

In [ ]:

# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
Train_Dataset_after_merge.head()

In [ ]:
Train_Dataset_after_merge.shape

In [ ]:

# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
test_Dataset_after_merge.head()

In [ ]:
test_Dataset_after_merge.shape

In [ ]:
def differentcolumns(Train, Test):
    L=[]
    for i in Test:
        if i not in Train:
            L.append(i)
    return(L)
Different_name=differentcolumns(Train_Dataset_after_merge,test_Dataset_after_merge)
print(Different_name)
type(Different_name)


In [ ]:
print(Different_name)
type(Different_name[0])


In [ ]:
for name in Different_name:
    #print(name)
    #type(name)
    l=name.split(sep='-')
    #print(l)
    name_new=str(l[0])+'_'+str(l[1])
    #print(name_new)
    test_Dataset_after_merge=test_Dataset_after_merge.rename(columns={name:name_new})
    
    
   

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
Columns_test_Dataset_after_merge=test_Dataset_after_merge.columns
print(Columns_test_Dataset_after_merge)

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
test_Dataset_after_merge.head()

In [ ]:

Different_name=differentcolumns(test_Dataset_after_merge,Train_Dataset_after_merge)
print(Different_name)
type(Different_name)

In [ ]:
percent_missing = [Train_Dataset_after_merge.isnull().sum() * 100 / len(Train_Dataset_after_merge)]
print(percent_missing)

In [ ]:
Train_Dataset_after_merge_Drop_col =Train_Dataset_after_merge.loc[:, (Train_Dataset_after_merge.isnull().sum() * 100 / len(Train_Dataset_after_merge)<60)]

In [ ]:
del Train_Dataset_after_merge
gc.collect()

In [ ]:
Train_Dataset_after_merge_Drop_col.head()
Train_Dataset_after_merge_Drop_col.shape

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
Train_Dataset_after_merge_Drop_col.head()

In [ ]:
cols=Train_Dataset_after_merge_Drop_col.columns
cols_list=cols.tolist()
#print(cols_list)
cols_list.remove('isFraud')

#print(cols_list)
        

In [ ]:
test_Dataset_after_merge_Drop_col =test_Dataset_after_merge.loc[:,cols_list]


In [ ]:
del(test_Dataset_after_merge)
gc.collect()

In [ ]:
test_Dataset_after_merge_Drop_col.shape

In [ ]:
print(cols_list)

In [ ]:
column_1 =Train_Dataset_after_merge_Drop_col["isFraud"]
column_2 = Train_Dataset_after_merge_Drop_col['ProductCD']
#a=column_1.corr(column_2)
#type(a)
print(Train_Dataset_after_merge_Drop_col['ProductCD'].dtypes)

In [ ]:
column_1 =Train_Dataset_after_merge_Drop_col["isFraud"]
column_2 = Train_Dataset_after_merge_Drop_col[i]
cols_drop=[]


for i in cols_list:
    print(i)
    column_2 = Train_Dataset_after_merge_Drop_col[i]
    print(type(column_1.corr(column_2)))
    #type(0.1)
    
    #if abs(round(float(column_1.corr(column_2)),2))<0.1:
        #cols_drop.append(i)
        
    

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
test_Dataset_after_merge_Drop_col.head()

In [ ]:
percent_missing = [Train_Dataset_after_merge_Drop_col.isnull().sum() * 100 / len(Train_Dataset_after_merge_Drop_col)]
print(percent_missing)


In [ ]:
Train_Dataset_after_merge_Drop_col.corr()['isFraud'].to_csv('/kaggle/working/df.csv')


In [ ]:
Train_Dataset_after_merge_Drop_col=Train_Dataset_after_merge_Drop_col.drop(labels=['V307','addr1','V317','V320','V214','V286','V294','V296','V41','V25','V104','card2','V130','V240','V65','V107','V297','V226','V14','V241','V321','V311','V298','V224','V223','V1','V318','V88','V319'], axis=1)
test_Dataset_after_merge_Drop_col=test_Dataset_after_merge_Drop_col.drop(labels=['V307','addr1','V317','V320','V214','V286','V294','V296','V41','V25','V104','card2','V130','V240','V65','V107','V297','V226','V14','V241','V321','V311','V298','V224','V223','V1','V318','V88','V319'], axis=1)
gc.collect()

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
Train_Dataset_after_merge_Drop_col.head()

In [ ]:
test_Dataset_after_merge_Drop_col.shape

In [ ]:
Train_Dataset_after_merge_Drop_col=Train_Dataset_after_merge_Drop_col.drop(labels=['V136','V137','V135','V299','V334','V305','V216','V210','V183','V280','V327','V295','V306','V128','V106','V175','V336','V127','V316','V133','V105','V134','V126','V279','V68','V95','V132','V97','V293',
'V101',
'V215',
'V102',
'V28',
'V27',
'V103',
'V268',
'V89',
'V266',
'V335',
'V96',
'V206',
'V337',
'V269',
'C3',
'V326',
'V267',
'id_05',
'D14',
'V285',
'V180',
'V205',
'V182',
'V328',
'C13',
'V207',
'V167',
'V99',
'V172',
'card1',
'V177',
'V339',
'V179',
'V168',
'V178',
'V202',
'V211',
'V209',
'V338',
'V234',
'id_13',
'V203',
'V213',
'V204',
'V212',
'V330',
'V173',
'V322',
'V331',
'V329',
'V323',
'V332',
'V324',
'V333',
'id_06',
'V164',
'D12',
'V166',
'V143',
'addr2',
'C5',
'C9',
'card5',
'V54',
'id_19',
'D9',
'D11',
'V13',
'D3',
'V76',
'V53',
'V165',
'V144',
'V75',
'V12',
'D6',
'V152',
'D13',
'id_04',], axis=1)

In [ ]:
gc.collect()

In [ ]:
Train_Dataset_after_merge_Drop_col.shape

In [ ]:
test_Dataset_after_merge_Drop_col=test_Dataset_after_merge_Drop_col.drop(labels=['V136','V137','V135','V299','V334','V305','V216','V210','V183','V280','V327','V295','V306','V128','V106','V175','V336','V127','V316','V133','V105','V134','V126','V279','V68','V95','V132','V97','V293',
'V101',
'V215',
'V102',
'V28',
'V27',
'V103',
'V268',
'V89',
'V266',
'V335',
'V96',
'V206',
'V337',
'V269',
'C3',
'V326',
'V267',
'id_05',
'D14',
'V285',
'V180',
'V205',
'V182',
'V328',
'C13',
'V207',
'V167',
'V99',
'V172',
'card1',
'V177',
'V339',
'V179',
'V168',
'V178',
'V202',
'V211',
'V209',
'V338',
'V234',
'id_13',
'V203',
'V213',
'V204',
'V212',
'V330',
'V173',
'V322',
'V331',
'V329',
'V323',
'V332',
'V324',
'V333',
'id_06',
'V164',
'D12',
'V166',
'V143',
'addr2',
'C5',
'C9',
'card5',
'V54',
'id_19',
'D9',
'D11',
'V13',
'D3',
'V76',
'V53',
'V165',
'V144',
'V75',
'V12',
'D6',
'V152',
'D13',
'id_04',], axis=1)

In [ ]:
gc.collect()

In [ ]:
pd.set_option("display.max_columns", None)
test_Dataset_after_merge_Drop_col.head()

In [ ]:
pd.set_option("display.max_rows", None)
display(Train_Dataset_after_merge_Drop_col[:100])

In [ ]:
 Train_Dataset_after_merge_Drop_col._get_numeric_data()

In [ ]:
# Get a Series object containing the data type objects of each column of Dataframe.
# Index of series is column name.
dataTypeSeries = Train_Dataset_after_merge_Drop_col.dtypes
print('Data type of each column of Dataframe :')
pd.set_option("display.max_rows", None)
print(dataTypeSeries)


In [ ]:
Train_obj_df = Train_Dataset_after_merge_Drop_col.select_dtypes(include=['object']).copy()
Train_obj_df.head()

In [ ]:
test_obj_df = test_Dataset_after_merge_Drop_col.select_dtypes(include=['object']).copy()
test_obj_df.head()

In [ ]:
Train_obj_df[Train_obj_df.columns]=Train_obj_df[Train_obj_df.columns].mask(pd.isnull, 'un')

In [ ]:
test_obj_df[test_obj_df.columns]=test_obj_df[test_obj_df.columns].mask(pd.isnull, 'un')

In [ ]:
test_obj_df.head()

In [ ]:
Train_obj_df.head()

In [ ]:
del Train_obj_df
gc.collect()

In [ ]:
L=[Train_obj_df[col].value_counts().to_dict() for col in Train_obj_df.columns ]

In [ ]:
L2=[test_obj_df[col].value_counts().to_dict() for col in test_obj_df.columns ]

In [ ]:
print(L2)

In [ ]:
print(L)

In [ ]:
Train_obj_df = Train_Dataset_after_merge_Drop_col.select_dtypes(include=['object']).copy()
Train_obj_df.head()
Train_obj_df[Train_obj_df.columns]=Train_obj_df[Train_obj_df.columns].mask(pd.isnull, 'un')
L=[Train_obj_df[col].value_counts().to_dict() for col in Train_obj_df.columns ]
cols_cat=[cols for cols in Train_obj_df.columns ]
for i in range(len(Train_obj_df.columns)):
    #print(i)
    Train_obj_df[cols_cat[i]]=Train_obj_df[cols_cat[i]].map(L[i])
    
Train_Dataset_after_merge_Drop_col=Train_Dataset_after_merge_Drop_col.drop(Train_obj_df.columns, axis=1)

In [ ]:
cols_cat=[cols for cols in Train_obj_df.columns ]
#print(cols_cat)

In [ ]:
cols_cat2=[cols for cols in test_obj_df.columns ]
#print(cols_cat)

In [ ]:
print(cols_cat[0])
print(L[0])

In [ ]:
for i in range(len(test_obj_df.columns)):
    #print(i)
    test_obj_df[cols_cat2[i]]=test_obj_df[cols_cat2[i]].map(L2[i])

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head

test_obj_df.head()

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head

Train_obj_df.head()

In [ ]:
Train_Dataset_after_merge_Drop_col=Train_Dataset_after_merge_Drop_col.drop(Train_obj_df.columns, axis=1)

In [ ]:
test_Dataset_after_merge_Drop_col=test_Dataset_after_merge_Drop_col.drop(test_obj_df.columns, axis=1)

In [ ]:
test_Dataset_after_merge_Drop_col.shape

In [ ]:
Train_Dataset_after_merge_Drop_col.shape

In [ ]:
Train_Dataset_after_merge_Drop_col = pd.concat([Train_Dataset_after_merge_Drop_col,Train_obj_df], axis=1)

In [ ]:
test_Dataset_after_merge_Drop_col = pd.concat([test_Dataset_after_merge_Drop_col,test_obj_df], axis=1)

In [ ]:
pd.set_option("display.max_columns", None)
Train_Dataset_after_merge_Drop_col.head()

In [ ]:
pd.set_option("display.max_columns", None)
test_Dataset_after_merge_Drop_col.head()

In [ ]:
test_Dataset_after_merge_Drop_col[test_Dataset_after_merge_Drop_col.columns]=test_Dataset_after_merge_Drop_col[test_Dataset_after_merge_Drop_col.columns].mask(pd.isnull,0)

In [ ]:
Train_Dataset_after_merge_Drop_col[Train_Dataset_after_merge_Drop_col.columns]=Train_Dataset_after_merge_Drop_col[Train_Dataset_after_merge_Drop_col.columns].mask(pd.isnull,0)

In [ ]:
pd.set_option("display.max_columns", None)
Train_Dataset_after_merge_Drop_col.head()

In [ ]:
Train_Dataset_after_merge_Drop_col.shape

In [ ]:

Train_Dataset_after_merge_Drop_col['isFraud'].value_counts().to_dict()

In [ ]:
test_Dataset_after_merge_Drop_col.shape

In [ ]:
Y=Train_Dataset_after_merge_Drop_col['isFraud']


In [ ]:
X=Train_Dataset_after_merge_Drop_col.drop('isFraud', axis=1)

In [ ]:
X.head()

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
oversample = SMOTE()
X, Y= oversample.fit_resample(X, Y)

In [ ]:
from collections import Counter
counter = Counter(Y)
print(counter)

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(X, Y)

In [ ]:
Y_predict=model.predict_proba(test_Dataset_after_merge_Drop_col)

In [ ]:
pd.set_option("display.max_rows", None)
print(Y_predict)

In [ ]:
sample_submission=pd.read_csv("/kaggle/input/ieee-fraud-detection/sample_submission.csv")

In [ ]:
del submission

In [ ]:
# Tạo DataFrame lưu kết quả
submission = pd.DataFrame({
    'TransactionID' : test_Dataset_after_merge_Drop_col.TransactionID,
    'isFraud' : (Y_predict[:,1])
})
submission.head()

In [ ]:
import 

In [ ]:
submission.to_csv('submission.csv', index=False)